In [1]:
import requests
import json
import pandas as pd
import os
import plotly.express as px
import numpy as np

# 1. Trazendo Dados da API

In [77]:
## Trazendo as informações da api

def fetch_data(filters={}):

  resultado = [] # Criação de um discionário para receber o json da página
  count = 0

  while(True):

    url = (f'https://api.obrasgov.gestao.gov.br/obrasgov/api/projeto-investimento?uf=DF&pagina={count}')

    #Armazena todas as informações da página em um objeto response
    response = requests.get(url, params=filters)

    if response.status_code == 200:
      # decodificação dos dados json e atribuição a uma variável.
      data = response.json()
      if data and data['content']: # Check if data and its content are not None or empty
          print(f"Adicionando dados na iteração nº: {count}")
          resultado.extend(data['content']) # Use extend to add elements from the list
      else:
          print("Parando: Nenhum conteúdo na página ou erro no endpoint da api")
          break # Exit the loop if no content is returned

    else:
      print(f"Erro na iteração nº: {count} com status code: {response.status_code}")
      break

    # if count == 50:
    #    break
    
    count+= 1

  if resultado:

    print("Salvando dados da API!")

    with open('cache_teste_api.json', 'w', encoding= 'utf-8') as f:

        json.dump(resultado, f, ensure_ascii=False, indent=4)
  else:
      print("Nenhum dado retornado da API!")

  return resultado

In [2]:
if os.path.exists('cache_teste_api.json'):
    
    with open('cache_teste_api.json', 'r', encoding= 'utf-8') as f:

        dados = json.load(f)
else:
    dados = fetch_data()

In [3]:
df = pd.DataFrame(dados)

# 2. Retirando dados agrupados em colunas com arrays

In [4]:
df['tomadorNome'] = df['tomadores'].apply(lambda x: x[0]['nome'] if len(x) > 0 else None)
df['tomadorCodigo'] = df['tomadores'].apply(lambda x: x[0]['codigo'] if len(x) > 0 else None)

In [5]:
df['executorNome'] = df['executores'].apply(lambda x: x[0]['nome'] if len(x) > 0 else None)
df['executorCodigo'] = df['executores'].apply(lambda x: x[0]['codigo'] if len(x) > 0 else None)

In [6]:
df['repassadorNome'] = df['repassadores'].apply(lambda x: x[0]['nome'] if len(x) > 0 else None)
df['repassadorCodigo'] = df['repassadores'].apply(lambda x: x[0]['codigo'] if len(x) > 0 else None)

In [7]:
df['origemFontesDeRecurso'] = df['fontesDeRecurso'].apply(lambda x: x[0]['origem'] if len(x) > 0 else None)
df['valorInvestimentoPrevisto'] = df['fontesDeRecurso'].apply(lambda x: x[0]['valorInvestimentoPrevisto'] if len(x) > 0 else None)

In [8]:
df_principal = df.drop(columns=['tomadores','executores','repassadores','fontesDeRecurso'])

In [9]:
df_principal.tail()

,idUnico,nome,cep,endereco,descricao,funcaoSocial,metaGlobal,dataInicialPrevista,dataFinalPrevista,dataInicialEfetiva,...,tipos,subTipos,tomadorNome,tomadorCodigo,executorNome,executorCodigo,repassadorNome,repassadorCodigo,origemFontesDeRecurso,valorInvestimentoPrevisto
495,43191.53-59,CONSTRUÇÃO DE UNIDADE DE ATENÇÃO ESPECIALIZADA...,72600-500,SETOR HOSPITALAR,CONSTRUÇÃO DE UNIDADE DE ATENÇÃO ESPECIALIZADA...,CONSTRUÇÃO DE UNIDADE DE ATENÇÃO ESPECIALIZADA...,CONSTRUÇÃO DE UNIDADE DE ATENÇÃO ESPECIALIZADA...,2017-03-29,2017-10-19,None,...,"[{'id': 39, 'descricao': 'Saúde', 'idEixo': 4}]","[{'id': 78, 'descricao': 'CAPS', 'idTipo': 39}]",FUNDO DE SAUDE DO DISTRITO FEDERAL,1.211625e+13,FUNDO DE SAUDE DO DISTRITO FEDERAL,1.211625e+13,FUNDO NACIONAL DE SAUDE,5.304930e+11,Federal,1312000.00
496,87956.53-43,Construção de Escola Classe (EC) no Jardim Man...,None,None,Construção de Escola Classe (EC) no Jardim Man...,Construção de Escola Classe (EC) no Jardim Man...,Escola - Projeto Próprio,2022-10-20,2026-05-12,None,...,"[{'id': 22, 'descricao': 'Infraestrutura Hídri...","[{'id': 36, 'descricao': 'Dragagem, Derrocamen...",None,NaN,FUNDO NACIONAL DE DESENVOLVIMENTO DA EDUCAÇÃO,2.530000e+02,None,NaN,Federal,0.01
497,2894.53-64,Contratação de empresa especializada na obra d...,1,"QGEx - Bloco E - 2º PISO, SMU",Contratação de empresa especializada na obra d...,Em relação a utilização de pesquisa de preços ...,Projeto de Adequação do Bloco E,2020-09-09,2021-11-03,None,...,"[{'id': 5, 'descricao': 'Administrativo', 'idE...","[{'id': 59, 'descricao': 'Obras em Imóveis de ...",COMANDO DO EXÉRCITO,9.400000e+01,COMANDO DO EXÉRCITO,9.400000e+01,MINISTÉRIO DA DEFESA,4.106600e+04,Federal,4848758.38
498,58219.53-55,Construção do Centro Público de Convivência - ...,None,None,Construção do Centro Público de Convivência,Promover a socialização e o fortalecimento de ...,Promover a socialização e o fortalecimento de ...,2025-06-01,2027-06-01,None,...,"[{'id': 6, 'descricao': 'Assistência Social', ...","[{'id': 97, 'descricao': 'Unidade de Acolhimen...",None,NaN,MINISTÉRIO DO DESENVOLVIMENTO E ASSISTÊNCIA SO...,3.087960e+05,None,NaN,Federal,684000.00
499,58220.53-25,Construção do Centro Público de Convivência - ...,None,None,Construção do Centro Público de Convivência - CC,Promover a socialização e o fortalecimento de ...,Promover a socialização e o fortalecimento de ...,2025-06-01,2027-06-01,None,...,"[{'id': 6, 'descricao': 'Assistência Social', ...","[{'id': 97, 'descricao': 'Unidade de Acolhimen...",None,NaN,MINISTÉRIO DO DESENVOLVIMENTO E ASSISTÊNCIA SO...,3.087960e+05,None,NaN,Federal,1070000.00


# 3. Plotando Gráfico - Barras

In [10]:
df_agrupado_natureza = df_principal.groupby('natureza').agg({ 'valorInvestimentoPrevisto': np.sum }).reset_index()

/tmp/ipykernel_790/896261810.py:1: FutureWarning: The provided callable <function sum at 0x7e4ca36d7d80> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df_agrupado_natureza = df_principal.groupby('natureza').agg({ 'valorInvestimentoPrevisto': np.sum }).reset_index()


In [11]:
df_agrupado_natureza

,natureza,valorInvestimentoPrevisto
0,Estudo,6.297713e+08
1,Obra,7.280313e+09
2,Outros,4.572423e+08
3,Projeto,2.818014e+08
4,Projeto de Investimento em Infraestrutura,6.036808e+08


In [12]:
# Apresentando gastos por tipo de investimento

px.bar(df_agrupado_natureza,"natureza","valorInvestimentoPrevisto")

# 4. Gráfico de Dispersão

In [108]:
df_agrupado_tomador = df_principal.groupby('tomadorNome').agg({ 'valorInvestimentoPrevisto': np.sum }).reset_index()

/tmp/ipykernel_10466/3762032958.py:1: FutureWarning:

The provided callable <function sum at 0x748b3cd03d80> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.



In [109]:
px.bar(df_agrupado_tomador, 'tomadorNome', 'valorInvestimentoPrevisto')

# 5. Gráfico de Pizza

In [97]:
df_quantidade_tomadores = df_principal['tomadorNome'].value_counts()

In [99]:
df_quantidade_tomadores.index

Index(['INSTITUTO FED. ED. CIENCIA E TEC. DE BRASILIA', 'COMANDO DO EXÉRCITO',
       'Polícia Militar do Distrito Federal',
       'FUNDO DE SAUDE DO DISTRITO FEDERAL',
       'FUND.UNIVERSIDADE FEDERAL VALE SAO FRANCISCO',
       'EMPRESA BRASILEIRA DE PESQUISA AGROPECUÁRIA',
       'DEPARTAMENTO NACIONAL DE INFRAESTRUTURA DE TRANSPORTES',
       'FUNDO DE ADMINISTRACAO DO HFA',
       'SUPERINTENDÊNCIA DA POLÍCIA RODOVIÁRIA FEDERAL NO DISTRITO FEDERAL',
       'COMANDO DA AERONÁUTICA', 'POLICIA CIVIL DO DISTRITO FEDERAL',
       'BANCO CENTRAL DO BRASIL-ORC.FISCAL/SEG.SOCIAL',
       'DEPARTAMENTO DE POLICIA FEDERAL', 'COMANDO DA MARINHA',
       'CORPO DE BOMBEIROS MILITAR DO DISTRITO FEDERAL',
       'PRESIDÊNCIA DA REPÚBLICA', 'EMPRESA PRIVADA',
       'FUNDO PENITENCIARIO NACIONAL', 'MINISTÉRIO DO MEIO AMBIENTE',
       'SECRETARIA DO TESOURO NACIONAL - STN',
       'Agência Brasileira de Inteligência', 'BANCO CENTRAL DO BRASIL',
       'DISTRITO FEDERAL', 'FUNDACAO UNIVERSIDADE

In [103]:
px.pie(df_quantidade_tomadores, names = df_quantidade_tomadores.index, values= df_quantidade_tomadores.values)